# Exploratory Data Analysis
The purpose for this notebook is to gain insights into the data and to understand it.

In [7]:
%pylab inline
import numpy as np
import pandas as pd
from os.path import join

Populating the interactive namespace from numpy and matplotlib


In [10]:
DATAPATH = '../data'
#df = pd.read_csv(join(DATAPATH, 'loan.csv'))
xl = pd.ExcelFile(join(DATAPATH, 'LCDataDictionary.xlsx'))

loan_stats   = xl.parse(xl.sheet_names[0])
browse_notes = xl.parse(xl.sheet_names[1])
reject_stats = xl.parse(xl.sheet_names[2])
del xl

## Initial data inspection
Here we will look into relationships between different data pieces and fundamental qualities such as existence of _missing_ data, _outliers_ and any potiential _inconsistencies_.

### Loan status `loan_stats`

In [12]:
loan_stats.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [38]:
print("Missing data: {} rows out of {} rows in total.".format(len(loan_stats) - loan_stats.dropna().shape[0], len(loan_stats)))
print("Are all values in `LoanStatNew` unique? {}".format(len(loan_stats['LoanStatNew'].unique()) == len(loan_stats)))
print("Are all values in `Description` unique? {}".format(len(loan_stats['Description'].unique()) == len(loan_stats)))

Missing data: 0 rows out of 151 rows in total.
Are all values in `LoanStatNew` unique? True
Are all values in `Description` unique? True


In [46]:
# basic data cleaning
loan_stats = loan_stats.dropna()
loan_stats['LoanStatNew'] = loan_stats['LoanStatNew'].str.replace(' ', '')
loan_stats['LoanStatNew'] = loan_stats['LoanStatNew'].str.lower()

print(loan_stats.to_string()) # <- uncomment to display all the values.

                                    LoanStatNew                                        Description
0                                acc_now_delinq  The number of accounts on which the borrower i...
1                          acc_open_past_24mths         Number of trades opened in past 24 months.
2                                    addr_state  The state provided by the borrower in the loan...
3                                      all_util              Balance to credit limit on all trades
4                                    annual_inc  The self-reported annual income provided by th...
5                              annual_inc_joint  The combined self-reported annual income provi...
6                              application_type  Indicates whether the loan is an individual ap...
7                                   avg_cur_bal            Average current balance of all accounts
8                                bc_open_to_buy          Total open to buy on revolving bankcards.
9         

The `loan_stats` is an auxiliary table that provides information about possible loan status (codes). It possesses good descriptions, but understanding it deeply requires domain expertise.
What is important at this stage is that we have:
* 151 **unique** codes, which are all delimited with an _underscore_ and written in small letters.

Perhaps that should be a convention?

### Browse Notes `browse_notes`

In [41]:
browse_notes.head()

,BrowseNotesFile,Description
0,acceptD,The date which the borrower accepted the offer
1,accNowDelinq,The number of accounts on which the borrower i...
2,accOpenPast24Mths,Number of trades opened in past 24 months.
3,addrState,The state provided by the borrower in the loan...
4,all_util,Balance to credit limit on all trades


In [44]:
print("Missing data: {} rows out of {} rows in total.".format(len(browse_notes) - browse_notes.dropna().shape[0], len(browse_notes)))
print("Are all values in `LoanStatNew` unique? {}".format(len(browse_notes['BrowseNotesFile'].unique()) == len(browse_notes)))
print("Are all values in `Description` unique? {}".format(len(browse_notes['Description'].unique()) == len(browse_notes)))

Missing data: 0 rows out of 120 rows in total.
Are all values in `LoanStatNew` unique? True
Are all values in `Description` unique? True


In [43]:
browse_notes = browse_notes.dropna()

In [112]:
import re

def camel2snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

browse_notes = browse_notes.copy()
browse_notes['BrowseNotesFile'] = browse_notes['BrowseNotesFile'].apply(lambda x: camel2snake(x))

In [120]:
print("Code words in `loan_stats` that overlap with the `browse_notes`: {}".format(
    len(set(browse_notes['BrowseNotesFile']).intersection(set(loan_stats['LoanStatNew'])))))

Code words in `loan_stats` that overlap with the `browse_notes`: 88
